In [1]:
import sys
import numpy
import pandas
import matplotlib
import seaborn
import scipy
import sklearn
print('Python: {}'.format(sys.version))
print('Numpy: {}'.format(numpy.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Matplotlib: {}'.format(matplotlib.__version__))
print('Seaborn: {}'.format(seaborn.__version__))
print('Scipy: {}'.format(scipy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))

Python: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Numpy: 1.14.0
Pandas: 0.22.0
Matplotlib: 2.1.2
Seaborn: 0.8.1
Scipy: 1.0.0
Sklearn: 0.19.1


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load the dataset from kaggle's CreditCard csv file and data.world's cc.csv file using pandas
data = pd.read_csv('creditcard.csv')

In [5]:
print(data.columns)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [32]:
data = data.sample(frac=0.1, random_state = 1)
print(data.shape)
#print(data.describe())

(28, 31)


In [33]:
#data.hist(figsize = (20, 20))
#plt.show()

In [14]:
Fraud = data[data['Class'] == 1]
Valid = data[data['Class'] == 0]

outlier_fraction = len(Fraud)/float(len(Valid))
print(outlier_fraction)

print('Fraud/Misuse Cases: {}'.format(len(Fraud)))
print('Valid Transactions: {}'.format(len(Valid)))

0.0017234102419808666
Fraud/Misuse Cases: 49
Valid Transactions: 28432


In [34]:
corrmat = data.corr()
#fig = plt.figure(figsize = (12, 10))

#sns.heatmap(corrmat, vmax = .9, square = True)
#plt.show()

In [10]:
columns = data.columns.tolist()
columns = [c for c in columns if c not in ["Class"]]
target = "Class"

X = data[columns]
Y = data[target]
print(X.shape)
print(Y.shape)

(28481, 30)
(28481,)


In [23]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

state = 1


classifiers = {
    "Random Forest": IsolationForest(max_samples=len(X),
                                        contamination=outlier_fraction,
                                        random_state=state),
    "Nearest Neighbour": LocalOutlierFactor(
        n_neighbors=20,
        contamination=outlier_fraction)}

In [27]:
plt.figure(figsize=(9, 7))
n_outliers = len(Fraud)
for i, (clf_name, clf) in enumerate(classifiers.items()):
    if clf_name == "Nearest Neighbour":
        y_pred = clf.fit_predict(X)
        scores_pred = clf.negative_outlier_factor_
    else:
        clf.fit(X)
        scores_pred = clf.decision_function(X)
        y_pred = clf.predict(X)
    
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    
    n_errors = (y_pred != Y).sum()
    
    print('{}: {}'.format(clf_name, n_errors))
    print(accuracy_score(Y, y_pred))
    print(classification_report(Y, y_pred))

Random Forest: 71
0.99750711000316
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     28432
          1       0.28      0.29      0.28        49

avg / total       1.00      1.00      1.00     28481

Nearest Neighbour: 97
0.9965942207085425
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     28432
          1       0.02      0.02      0.02        49

avg / total       1.00      1.00      1.00     28481

